In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean
from collections import defaultdict
from sklearn.model_selection import GroupKFold
from sklearn.metrics import log_loss, accuracy_score
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from itertools import product
from scipy.stats import zscore
import plotly.express as px
from scipy.optimize import minimize
import seaborn as sns

# Load dataset
print("Loading dataset...")
data = pd.read_csv(r'D:\University of Rochester\Fall24\RA - Florian\Code\Li_Xie_2020_L1_vowels_statistics_general.csv')
print("Dataset loaded successfully.")

# Compute z-scores for F1 and F2
print("Computing z-scores for F1 and F2...")
data['F1_z'] = zscore(data['F1_gm'])
data['F2_z'] = zscore(data['F2_gm'])
print("Z-scores computed.")

# Define sigma (fixed noise sigma for optimization)
fixed_sigma = 1.0  # Fixed noise sigma

# Free parameters for distance and similarity
distance_exponent = 2  # Exponent of the distance

# Feature indices for F1 and F2 (z-scored)
feature_indices = ['F1_z', 'F2_z']  # Using standardized F1 and F2

# Add noise to features
def add_noise(features, noise_level):
    # Ensure the scale parameter is non-negative by taking the absolute value of the feature
    return [f + np.random.normal(0, noise_level * abs(f)) for f in features]

# Compute weighted Euclidean distance with exponent
def compute_distance(input_features, exemplar_features, weights, exponent):
    weighted_diff = weights * (np.array(input_features) - np.array(exemplar_features))
    distance = np.sum(np.abs(weighted_diff) ** exponent) ** (1/exponent)
    return distance

# Compute similarity using Gaussian function with scaling factor
def compute_similarity(input_features, exemplar_features, weights, exponent, scaling, sigma):
    dist = compute_distance(input_features, exemplar_features, weights, exponent)
    return np.exp(- (scaling * dist) ** 2 / (2 * sigma ** 2))

# Predict posterior probabilities based on exemplars
def predict_posterior(input_features, exemplars, weights, exponent, scaling, sigma, prior):
    category_similarities = {}
    total_similarity = 0
    for category, ex_list in exemplars.items():
        sim_sum = sum(compute_similarity(input_features, ex, weights, exponent, scaling, sigma) for ex in ex_list)
        category_similarities[category] = sim_sum
        total_similarity += sim_sum
    if total_similarity == 0:
        return None
    # Apply prior
    posteriors = {cat: (sim_sum * prior.get(cat, 1.0)) for cat, sim_sum in category_similarities.items()}
    total_posterior = sum(posteriors.values())
    # Avoid division by zero
    if total_posterior == 0:
        return None
    posteriors = {cat: (sim / total_posterior) for cat, sim in posteriors.items()}
    return posteriors

# Cross-validation setup
print("Setting up cross-validation...")
group_kfold = GroupKFold(n_splits=5)  # Reduced from 17 to 5 for better fold sizes
groups = data['Talker']
print("Cross-validation setup complete.")

# Define uniform prior
unique_categories = sorted(data['Vowel'].unique())
prior = {cat: 1.0 for cat in unique_categories}  # Uniform prior
print(f"Unique categories: {unique_categories}")

# Function to calculate log likelihood for a single prediction
def calculate_log_likelihood(true_label, posterior, classes):
    if posterior is None:
        return 0
    prob = posterior.get(true_label, 1e-10)
    # Ensure probability is not zero or negative
    prob = max(prob, 1e-10)
    return np.log(prob)

# Function to perform cross-validation and evaluate the model
def evaluate_model(data, sigma, noise_level, group_kfold, groups, weights, exponent, scaling, prior):
    print(f"Evaluating model with sigma={sigma}, noise_level={noise_level}, weights={weights}, scaling={scaling}")
    log_likelihoods = []
    
    fold = 1
    for train_idx, test_idx in group_kfold.split(data, groups=groups):
        print(f"  Processing fold {fold}...")
        train_data, test_data = data.iloc[train_idx], data.iloc[test_idx]
        exemplars_train = defaultdict(list)
        
        # Train exemplars with added noise
        for _, row in train_data.iterrows():
            category = row['Vowel']
            features = row[feature_indices].tolist()
            noisy = add_noise(features, noise_level)
            if not np.any(np.isnan(noisy)) and not np.any(np.isinf(noisy)):
                exemplars_train[category].append(noisy)
        
        true_labels = []
        preds_posteriors = []
        
        # Predict on test set
        for _, row in test_data.iterrows():
            true = row['Vowel']
            input_feat = row[feature_indices].tolist()
            noisy_input = add_noise(input_feat, noise_level)
            if not np.any(np.isnan(noisy_input)) and not np.any(np.isinf(noisy_input)):
                posteriors = predict_posterior(noisy_input, exemplars_train, weights, exponent, scaling, sigma, prior)
                if posteriors:
                    preds_posteriors.append(posteriors)
                    true_labels.append(true)
        
        if true_labels:
            # Calculate log likelihood
            fold_log_likelihood = sum([calculate_log_likelihood(t, p, unique_categories) for t, p in zip(true_labels, preds_posteriors)])
            log_likelihoods.append(fold_log_likelihood)
            
            print(f"  Fold {fold} - Log Likelihood: {fold_log_likelihood:.4f}")
        else:
            print(f"  Fold {fold} - No valid predictions.")
        fold += 1
    
    # Aggregate results
    mean_ll = np.mean(log_likelihoods) if log_likelihoods else np.nan
    print(f"  Mean Log Likelihood: {mean_ll:.4f}")
    
    return mean_ll  # Return only mean log likelihood

# Combined objective function for optimization
def objective_function(params):
    weight_f1, weight_f2, scaling = params
    # Ensure weights sum to 1
    if not np.isclose(weight_f1 + weight_f2, 1.0):
        return np.inf  # Penalize if weights do not sum to 1
    # Ensure weights are within bounds (0 to 1)
    if not (0.0 <= weight_f1 <= 1.0) or not (0.0 <= weight_f2 <= 1.0):
        return np.inf
    # Ensure scaling is positive
    if scaling <= 0:
        return np.inf
    weights_array = np.array([weight_f1, weight_f2])
    mean_ll = evaluate_model(
        data, fixed_sigma, noise_level_fixed, group_kfold, groups, weights_array, distance_exponent, scaling, prior
    )
    # We aim to maximize log likelihood
    # Since optimization functions minimize, return the negative
    return -mean_ll

# Optimization bounds for weights (0 to 1) and similarity scaling (0 to inf)
bounds = [
    (0.0, 1.0),  # weight_F1
    (0.0, 1.0),  # weight_F2
    (1e-5, None)  # similarity_scaling, set lower bound to a small positive number to avoid division by zero
]

# Initial guess
initial_guess = [0.5, 0.5, 0]  # [weight_f1, weight_f2, similarity_scaling]

# Fixed noise level
noise_level_fixed = 0.0  # Fixed noise level set to 0

# Constraints: weights must sum to 1
constraints = {
    'type': 'eq',
    'fun': lambda x: x[0] + x[1] - 1.0  # weight_f1 + weight_f2 = 1
}

# Perform optimization using scipy.optimize.minimize with SLSQP method
print("Starting parameter optimization using scipy.optimize.minimize...")
result = minimize(
    objective_function,
    initial_guess,
    method='SLSQP',
    bounds=bounds,
    constraints=constraints,
    options={'disp': True, 'maxiter': 90}
)

if result.success:
    optimized_weight_f1, optimized_weight_f2, optimized_scaling = result.x
    print(f"\nOptimization successful!")
    print(f"Optimized Feature Weight F1_z: {optimized_weight_f1:.4f}")
    print(f"Optimized Feature Weight F2_z: {optimized_weight_f2:.4f}")
    print(f"Optimized Similarity Scaling: {optimized_scaling:.4f}")
else:
    print("\nOptimization failed.")
    print(result.message)

# ------------------ Grid Search for Heatmap ------------------

# Define fine-grained parameter ranges for heatmap
feature_weight_f1_grid = np.linspace(0.0, 1.0, 10)  # 10 steps for higher resolution
feature_weight_f2_grid = np.linspace(0.0, 1.0, 10)  # 10 steps for higher resolution
similarity_scaling_grid = np.linspace(0.5, 20, 10)  # Similarity scaling from 0.5 to 2.0

heatmap_results = []

print("\nStarting grid search for heatmap visualization...")
total_iterations = len(feature_weight_f1_grid) * len(feature_weight_f2_grid) * len(similarity_scaling_grid)
current_iteration = 1

for fw1 in feature_weight_f1_grid:
    for fw2 in feature_weight_f2_grid:
        # Ensure weights sum to 1
        if not np.isclose(fw1 + fw2, 1.0):
            continue
        for ss in similarity_scaling_grid:
            print(f"Evaluating combination {current_iteration}/{total_iterations}: F1 Weight={fw1:.2f}, F2 Weight={fw2:.2f}, Scaling={ss:.2f}")
            weights_array = np.array([fw1, fw2])
            mean_ll = evaluate_model(
                data, fixed_sigma, noise_level_fixed, group_kfold, groups, weights_array, distance_exponent, ss, prior
            )
            heatmap_results.append({
                'weight_f1': fw1,
                'weight_f2': fw2,
                'similarity_scaling': ss,
                'mean_log_likelihood': mean_ll
            })
            current_iteration += 1

# Convert heatmap_results to DataFrame
heatmap_df = pd.DataFrame(heatmap_results)

# ------------------ Visualization ------------------

# 3D Heatmap using Plotly
fig_3d = px.scatter_3d(
    heatmap_df,
    x='weight_f1',
    y='weight_f2',
    z='similarity_scaling',
    color='mean_log_likelihood',
    title='Mean Log Likelihood Variation with F1 Weight, F2 Weight, and Similarity Scaling',
    labels={
        'weight_f1': 'Feature Weight F1_z',
        'weight_f2': 'Feature Weight F2_z',
        'similarity_scaling': 'Similarity Scaling',
        'mean_log_likelihood': 'Mean Log Likelihood'
    },
    color_continuous_scale='Viridis'
)
fig_3d.show()

# # 2D Heatmaps for different similarity scaling levels
# plt.figure(figsize=(20, 16))
# for idx, scaling in enumerate(similarity_scaling_grid):
#     plt.subplot(4, 3, idx+1)
#     subset = heatmap_df[heatmap_df['similarity_scaling'] == scaling]
#     pivot_table = subset.pivot(index='weight_f2', columns='weight_f1', values='mean_log_likelihood')
#     sns.heatmap(pivot_table, cmap='viridis', annot=False, cbar=idx==0)
#     plt.title(f'Similarity Scaling = {scaling:.2f}')
#     plt.xlabel('Feature Weight F1_z')
#     plt.ylabel('Feature Weight F2_z')

# plt.tight_layout()
# plt.show()

Loading dataset...
Dataset loaded successfully.
Computing z-scores for F1 and F2...
Z-scores computed.
Setting up cross-validation...
Cross-validation setup complete.
Unique categories: ['aa1', 'ae1', 'ah1', 'eh1', 'ih1', 'iy1', 'uh1', 'uw1']
Starting parameter optimization using scipy.optimize.minimize...
Evaluating model with sigma=1.0, noise_level=0.0, weights=[0.5 0.5], scaling=1e-05
  Processing fold 1...
  Fold 1 - Log Likelihood: -467.8828
  Processing fold 2...
  Fold 2 - Log Likelihood: -465.8018
  Processing fold 3...
  Fold 3 - Log Likelihood: -455.3915
  Processing fold 4...
  Fold 4 - Log Likelihood: -598.8892
  Processing fold 5...
  Fold 5 - Log Likelihood: -596.7922
  Mean Log Likelihood: -516.9515
Evaluating model with sigma=1.0, noise_level=0.0, weights=[0.50000001 0.5       ], scaling=1e-05
  Processing fold 1...
  Fold 1 - Log Likelihood: -467.8828
  Processing fold 2...
  Fold 2 - Log Likelihood: -465.8018
  Processing fold 3...
  Fold 3 - Log Likelihood: -455.3915

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean
from collections import defaultdict
from sklearn.model_selection import GroupKFold
from sklearn.metrics import log_loss, accuracy_score
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from itertools import product
from scipy.stats import zscore
import plotly.express as px
from scipy.optimize import minimize
import seaborn as sns

# Load dataset
print("Loading dataset...")
data = pd.read_csv(r'D:\University of Rochester\Fall24\RA - Florian\Code\Li_Xie_2020_L1_vowels_statistics_general.csv')
data_listner = pd.read_csv(r"D:\University of Rochester\Fall24\RA - Florian\Code\Experiment-NORM-AB-after-exclusions.csv")
print("Datasets loaded successfully.")

# Compute z-scores for F1 and F2
print("Computing z-scores for F1 and F2...")
data['F1_z'] = zscore(data['F1_gm'])
data['F2_z'] = zscore(data['F2_gm'])
print("Z-scores computed.")

# Define sigma (fixed noise sigma for optimization)
fixed_sigma = 1.0  # Fixed noise sigma

# Free parameters for distance and similarity
distance_exponent = 2  # Exponent of the distance

# Feature indices for F1 and F2 (z-scored)
feature_indices = ['F1_z', 'F2_z']  # Using standardized F1 and F2

# Add noise to features
def add_noise(features, noise_level):
    # Ensure the scale parameter is non-negative by taking the absolute value of the feature
    return [f + np.random.normal(0, noise_level * abs(f)) for f in features]

# Compute weighted Euclidean distance with exponent
def compute_distance(input_features, exemplar_features, weights, exponent):
    weighted_diff = weights * (np.array(input_features) - np.array(exemplar_features))
    distance = np.sum(np.abs(weighted_diff) ** exponent) ** (1/exponent)
    return distance

# Compute similarity using Gaussian function with scaling factor
def compute_similarity(input_features, exemplar_features, weights, exponent, scaling, sigma):
    dist = compute_distance(input_features, exemplar_features, weights, exponent)
    return np.exp(- (scaling * dist) ** 2 / (2 * sigma ** 2))

# Predict posterior probabilities based on exemplars
def predict_posterior(input_features, exemplars, weights, exponent, scaling, sigma, prior):
    category_similarities = {}
    total_similarity = 0
    for category, ex_list in exemplars.items():
        sim_sum = sum(compute_similarity(input_features, ex, weights, exponent, scaling, sigma) for ex in ex_list)
        category_similarities[category] = sim_sum
        total_similarity += sim_sum
    if total_similarity == 0:
        return None
    # Apply prior
    posteriors = {cat: (sim_sum * prior.get(cat, 1.0)) for cat, sim_sum in category_similarities.items()}
    total_posterior = sum(posteriors.values())
    # Avoid division by zero
    if total_posterior == 0:
        return None
    posteriors = {cat: (sim / total_posterior) for cat, sim in posteriors.items()}
    return posteriors

# Cross-validation setup
print("Setting up cross-validation...")
group_kfold = GroupKFold(n_splits=5)  # Reduced from 17 to 5 for better fold sizes
groups = data['Talker']
print("Cross-validation setup complete.")

# Define uniform prior
unique_categories = sorted(data['Vowel'].unique())
prior = {cat: 1.0 for cat in unique_categories}  # Uniform prior
print(f"Unique categories: {unique_categories}")

# Function to calculate log likelihood for a single prediction
def calculate_log_likelihood(true_label, posterior, classes):
    if posterior is None:
        return 0
    prob = posterior.get(true_label, 1e-10)
    # Ensure probability is not zero or negative
    prob = max(prob, 1e-10)
    return np.log(prob)

# Function to perform cross-validation and evaluate the model
def evaluate_model(data, sigma, noise_level, group_kfold, groups, weights, exponent, scaling, prior):
    print(f"Evaluating model with sigma={sigma}, noise_level={noise_level}, weights={weights}, scaling={scaling}")
    log_likelihoods = []
    
    fold = 1
    for train_idx, test_idx in group_kfold.split(data, groups=groups):
        print(f"  Processing fold {fold}...")
        train_data, test_data = data.iloc[train_idx], data.iloc[test_idx]
        exemplars_train = defaultdict(list)
        
        # Train exemplars with added noise
        for _, row in train_data.iterrows():
            category = row['Vowel']
            features = row[feature_indices].tolist()
            noisy = add_noise(features, noise_level)
            if not np.any(np.isnan(noisy)) and not np.any(np.isinf(noisy)):
                exemplars_train[category].append(noisy)
        
        true_labels = []
        preds_posteriors = []
        
        # Predict on test set
        for _, row in test_data.iterrows():
            true = row['Vowel']
            input_feat = row[feature_indices].tolist()
            noisy_input = add_noise(input_feat, noise_level)
            if not np.any(np.isnan(noisy_input)) and not np.any(np.isinf(noisy_input)):
                posteriors = predict_posterior(noisy_input, exemplars_train, weights, exponent, scaling, sigma, prior)
                if posteriors:
                    preds_posteriors.append(posteriors)
                    true_labels.append(true)
        
        if true_labels:
            # Calculate log likelihood
            fold_log_likelihood = sum([calculate_log_likelihood(t, p, unique_categories) for t, p in zip(true_labels, preds_posteriors)])
            log_likelihoods.append(fold_log_likelihood)
            
            print(f"  Fold {fold} - Log Likelihood: {fold_log_likelihood:.4f}")
        else:
            print(f"  Fold {fold} - No valid predictions.")
        fold += 1
    
    # Aggregate results
    mean_ll = np.mean(log_likelihoods) if log_likelihoods else np.nan
    print(f"  Mean Log Likelihood: {mean_ll:.4f}")
    
    return mean_ll  # Return only mean log likelihood

# Combined objective function for optimization
def objective_function(params):
    weight_f1, weight_f2, scaling = params
    # Ensure weights sum to 1
    if not np.isclose(weight_f1 + weight_f2, 1.0):
        return np.inf  # Penalize if weights do not sum to 1
    # Ensure weights are within bounds (0 to 1)
    if not (0.0 <= weight_f1 <= 1.0) or not (0.0 <= weight_f2 <= 1.0):
        return np.inf
    # Ensure scaling is positive
    if scaling <= 0:
        return np.inf
    weights_array = np.array([weight_f1, weight_f2])
    mean_ll = evaluate_model(
        data, fixed_sigma, noise_level_fixed, group_kfold, groups, weights_array, distance_exponent, scaling, prior
    )
    # We aim to maximize log likelihood
    # Since optimization functions minimize, return the negative
    return -mean_ll

# Optimization bounds for weights (0 to 1) and similarity scaling (0 to inf)
bounds = [
    (0.0, 1.0),  # weight_F1
    (0.0, 1.0),  # weight_F2
    (1e-5, None)  # similarity_scaling, set lower bound to a small positive number to avoid division by zero
]

# Initial guess
initial_guess = [0.5, 0.5, 0]  # [weight_f1, weight_f2, similarity_scaling]

# Fixed noise level
noise_level_fixed = 0.0  # Fixed noise level set to 0

# Constraints: weights must sum to 1
constraints = {
    'type': 'eq',
    'fun': lambda x: x[0] + x[1] - 1.0  # weight_f1 + weight_f2 = 1
}

# Perform optimization using scipy.optimize.minimize with SLSQP method
print("Starting parameter optimization using scipy.optimize.minimize...")
result = minimize(
    objective_function,
    initial_guess,
    method='SLSQP',
    bounds=bounds,
    constraints=constraints,
    options={'disp': True, 'maxiter': 90}
)

if result.success:
    optimized_weight_f1, optimized_weight_f2, optimized_scaling = result.x
    print(f"\nOptimization successful!")
    print(f"Optimized Feature Weight F1_z: {optimized_weight_f1:.4f}")
    print(f"Optimized Feature Weight F2_z: {optimized_weight_f2:.4f}")
    print(f"Optimized Similarity Scaling: {optimized_scaling:.4f}")
else:
    print("\nOptimization failed.")
    print(result.message)

# ------------------ Grid Search for Heatmap ------------------

# Define fine-grained parameter ranges for heatmap
feature_weight_f1_grid = np.linspace(0.0, 1.0, 10)  # 10 steps for higher resolution
feature_weight_f2_grid = np.linspace(0.0, 1.0, 10)  # 10 steps for higher resolution
similarity_scaling_grid = np.linspace(0.5, 30, 15)  # Similarity scaling from 0.5 to 2.0

heatmap_results = []

print("\nStarting grid search for heatmap visualization...")
total_iterations = len(feature_weight_f1_grid) * len(feature_weight_f2_grid) * len(similarity_scaling_grid)
current_iteration = 1

for fw1 in feature_weight_f1_grid:
    for fw2 in feature_weight_f2_grid:
        # Ensure weights sum to 1
        if not np.isclose(fw1 + fw2, 1.0):
            continue
        for ss in similarity_scaling_grid:
            print(f"Evaluating combination {current_iteration}/{total_iterations}: F1 Weight={fw1:.2f}, F2 Weight={fw2:.2f}, Scaling={ss:.2f}")
            weights_array = np.array([fw1, fw2])
            mean_ll = evaluate_model(
                data, fixed_sigma, noise_level_fixed, group_kfold, groups, weights_array, distance_exponent, ss, prior
            )
            heatmap_results.append({
                'weight_f1': fw1,
                'weight_f2': fw2,
                'similarity_scaling': ss,
                'mean_log_likelihood': mean_ll
            })
            current_iteration += 1

# Convert heatmap_results to DataFrame
heatmap_df = pd.DataFrame(heatmap_results)

# ------------------ Visualization ------------------

# 3D Heatmap using Plotly
fig_3d = px.scatter_3d(
    heatmap_df,
    x='weight_f1',
    y='weight_f2',
    z='similarity_scaling',
    color='mean_log_likelihood',
    title='Mean Log Likelihood Variation with F1 Weight, F2 Weight, and Similarity Scaling',
    labels={
        'weight_f1': 'Feature Weight F1_z',
        'weight_f2': 'Feature Weight F2_z',
        'similarity_scaling': 'Similarity Scaling',
        'mean_log_likelihood': 'Mean Log Likelihood'
    },
    color_continuous_scale='Viridis'
)
fig_3d.show()

# # 2D Heatmaps for different similarity scaling levels
# plt.figure(figsize=(20, 16))
# for idx, scaling in enumerate(similarity_scaling_grid):
#     plt.subplot(4, 3, idx+1)
#     subset = heatmap_df[heatmap_df['similarity_scaling'] == scaling]
#     pivot_table = subset.pivot(index='weight_f2', columns='weight_f1', values='mean_log_likelihood')
#     sns.heatmap(pivot_table, cmap='viridis', annot=False, cbar=idx==0)
#     plt.title(f'Similarity Scaling = {scaling:.2f}')
#     plt.xlabel('Feature Weight F1_z')
#     plt.ylabel('Feature Weight F2_z')

# plt.tight_layout()
# plt.show()

Loading dataset...
Dataset loaded successfully.
Computing z-scores for F1 and F2...
Z-scores computed.
Setting up cross-validation...
Cross-validation setup complete.
Unique categories: ['aa1', 'ae1', 'ah1', 'eh1', 'ih1', 'iy1', 'uh1', 'uw1']
Starting parameter optimization using scipy.optimize.minimize...
Evaluating model with sigma=1.0, noise_level=0.0, weights=[0.5 0.5], scaling=1e-05
  Processing fold 1...
  Fold 1 - Log Likelihood: -467.8828
  Processing fold 2...
  Fold 2 - Log Likelihood: -465.8018
  Processing fold 3...
  Fold 3 - Log Likelihood: -455.3915
  Processing fold 4...
  Fold 4 - Log Likelihood: -598.8892
  Processing fold 5...
  Fold 5 - Log Likelihood: -596.7922
  Mean Log Likelihood: -516.9515
Evaluating model with sigma=1.0, noise_level=0.0, weights=[0.50000001 0.5       ], scaling=1e-05
  Processing fold 1...
  Fold 1 - Log Likelihood: -467.8828
  Processing fold 2...
  Fold 2 - Log Likelihood: -465.8018
  Processing fold 3...
  Fold 3 - Log Likelihood: -455.3915